<a href="https://colab.research.google.com/github/kirill-kozlitin/Coworking/blob/main/%D0%9A%D0%BE%D0%B2%D0%BE%D1%80%D0%BA%D0%B8%D0%BD%D0%B3_%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D0%BA%D0%BE%D0%BC%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D1%80%D0%B8%D0%B5%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Импорт бибилиотек

In [1]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
#

# # Выведите результат
# for combination in all_combinations:
#     print(combination)
# Этот код создаст все возможные комбинации
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Загрузка комментариев

In [3]:
path = '/content/drive/MyDrive/Коворкинг/Выборки/'
data = pd.read_excel(path + 'Комментарии.xlsx')

#### Предобработка

In [4]:
data[['Название', 'Адрес']] = data[['Название', 'Адрес']].ffill()

In [5]:
data.head()

,Название,Адрес,Отзыв,Оценка
0,& Place,"Невский проспект, 71/1","Место, сделанное людьми для людей. Удобное рас...",5.00
1,& Place,"Невский проспект, 71/1","Обожаю это место. Живу не в питере, но в после...",5.00
2,& Place,"Невский проспект, 71/1",Суперкомфоотный коворкинг для работы! Уже боль...,5.00
3,& Place,"Невский проспект, 71/1",Супер место. Чистота везде и всегда. Кофе - де...,5.00
4,& Place,"Невский проспект, 71/1","Выбирали из множества вариантов, но этот ковор...",5.00


#### Аналитика

In [6]:
from transformers import DistilBertTokenizer, DistilBertModel

In [7]:
import torch

In [8]:
# Загрузка токенизатора и модели DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
model = DistilBertModel.from_pretrained('distilbert-base-multilingual-cased')

In [9]:
# Обновление функции encode_text для использования DistilBERT
def encode_text(texts):
    # Токенизация текстов
    inputs = tokenizer(texts, return_tensors='pt', truncation=True, padding=True, max_length=256)

    # Пропускание токенизированного текста через модель DistilBERT
    outputs = model(**inputs)

    # Извлечение векторных представлений текста (последний слой скрытых состояний)
    embeddings = outputs.last_hidden_state

    # Усреднение по всем токенам для получения векторного представления всего текста
    text_vectors = torch.mean(embeddings, dim=1)

    return text_vectors

In [10]:
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
# Размер батча
batch_size = 4

# Создание DataLoader для батчевой обработки
data_loader = DataLoader(data['Отзыв'].tolist(), batch_size=batch_size)

review_vectors = []

# Обработка отзывов батчами и сохранение результатов
for batch in tqdm(data_loader, desc="Processing batches"):
    batch_vectors = encode_text(batch)
    review_vectors.extend(batch_vectors)



Processing batches:  46%|████▌     | 53/116 [00:40<00:41,  1.52it/s]

In [ ]:
# Сохранение векторных представлений в DataFrame
# df['review_vectors'] = review_vectors



In [ ]:
data_loader